## Content-Based Recommenders

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip "/content/drive/MyDrive/Colab_Notebooks/Year_4/Recommendation_Engines/yelp_restaurant_pickles.zip" -d /content/

Archive:  /content/drive/MyDrive/Colab_Notebooks/Year_4/Recommendation_Engines/yelp_restaurant_pickles.zip
 extracting: /content/reviews_sampled.pkl  
 extracting: /content/user_sampled.pkl  
 extracting: /content/business_sampled.pkl  
 extracting: /content/checkin_sampled.pkl  
 extracting: /content/tip_sampled.pkl  


In [3]:
import pandas as pd
reviews_sampled = pd.read_pickle("reviews_sampled.pkl")

In [4]:
user_sampled = pd.read_pickle("user_sampled.pkl")

In [5]:
business_sampled = pd.read_pickle("business_sampled.pkl")

In [6]:
checkin_sampled = pd.read_pickle("checkin_sampled.pkl")

In [7]:
tip_sampled = pd.read_pickle("tip_sampled.pkl")

### BERT

In [8]:
reviews_df = pd.read_pickle("reviews_sampled.pkl")
business_df = pd.read_pickle("business_sampled.pkl")

# Get unique business IDs from reviews
business_ids = reviews_df['business_id'].unique().tolist()
print(f"Loaded {len(business_ids)} businesses")

Loaded 14030 businesses


In [9]:
aggregated_reviews = reviews_df.groupby('business_id')['text'].apply(
    lambda x: " ".join(x)[:5000]  # Truncate to avoid memory issues
)
print("Reviews aggregated for", len(aggregated_reviews), "businesses")

Reviews aggregated for 14030 businesses


In [10]:
!pip install -q sentence-transformers
from sentence_transformers import SentenceTransformer
import numpy as np

# lighter model used here
model = SentenceTransformer('all-MiniLM-L6-v2')
print("Model loaded - ready for encoding!")

def batch_encode(texts, batch_size=64):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        embeddings.append(model.encode(batch, show_progress_bar=True))
        print(f"Processed {min(i+batch_size, len(texts))}/{len(texts)} batches")
    return np.vstack(embeddings)

review_texts = aggregated_reviews.tolist()
embeddings = batch_encode(review_texts)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━ 479.4/664.8 MB 123.4 MB/s eta 0:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.6 MB/s eta 0:00:00
ERROR: THESE PACKAGES DO NOT MATCH THE HASHES FROM THE REQUIREMENTS FILE. If you have updated the package versions, please update the hashes. Otherwise, examine the package contents carefully; someone 

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

Processed 13952/14030 batches


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Processed 14016/14030 batches


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Processed 14030/14030 batches


In [11]:
embeddings_df = pd.DataFrame(embeddings, index=aggregated_reviews.index)
print("Embeddings shape:", embeddings_df.shape)

Embeddings shape: (14030, 384)


### Similarities

#### **Similarity Matrix**

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
bert_sim_matrix = cosine_similarity(embeddings_df)
bert_sim_df = pd.DataFrame(
    bert_sim_matrix,
    index=embeddings_df.index,
    columns=embeddings_df.index
)
print("BERT similarity matrix computed!")

BERT similarity matrix computed!


#### **Coverage**

In [14]:
def calculate_bert_coverage(threshold=0.5):
    coverage = (bert_sim_df.max(axis=1) > threshold).mean() * 100
    print(f"BERT Coverage (@{threshold}): {coverage:.1f}%")
    return coverage

# Calculate at different thresholds
coverage_results = {
    '0.3': calculate_bert_coverage(0.3),
    '0.5': calculate_bert_coverage(0.5),
    '0.7': calculate_bert_coverage(0.7)
}

BERT Coverage (@0.3): 100.0%
BERT Coverage (@0.5): 100.0%
BERT Coverage (@0.7): 100.0%


#### **Diversity**

In [15]:
def calculate_bert_diversity(top_n=5):
    """Measure how dissimilar top recommendations are to each other"""
    top_recs = bert_sim_df.apply(lambda x: x.nlargest(top_n+1).index[1:], axis=1)
    diversities = []
    for biz in top_recs.index:
        rec_ids = top_recs[biz]
        sub_matrix = bert_sim_df.loc[rec_ids, rec_ids]
        diversities.append(1 - sub_matrix.values.mean()) # 1 - average similarity
    return np.mean(diversities)

print(f"\nBERT Diversity Score: {calculate_bert_diversity():.3f} (1 = most diverse)")



BERT Diversity Score: 0.254 (1 = most diverse)


#### **Recommendation Function**

In [16]:
def get_bert_recommendations(business_id, top_n=5, business_df=business_df):
    """Get top_n recommendations for a business using BERT embeddings"""
    if business_id not in bert_sim_df.index:
        return pd.DataFrame()  # Return empty if business not found

    sim_scores = bert_sim_df[business_id].sort_values(ascending=False)[1:top_n+1]
    recommendations = business_df.loc[sim_scores.index, ['name', 'categories', 'city', 'stars']]
    recommendations['similarity_score'] = sim_scores.values
    return recommendations

In [17]:
example_biz = business_df.index[0]  # Replace with any business ID
print(f"\nExample recommendations for business: {business_df.loc[example_biz, 'name']}")
display(get_bert_recommendations(example_biz))


Example recommendations for business: St Honore Pastries


""
